In [22]:
import numpy as np
import plotly.graph_objects as go

# scipy 
from scipy.signal import find_peaks
from scipy.optimize import curve_fit

# utils and helper_files
from utils.get_raw_data import get_multiple_data_arrays
from helper_files.gaussian_fitting import gaussian, n_gaussians

In [23]:
# parameters

deg = 12 # degree of the polynomial to the background
prominence = 0.01 # prominence of the peaks, for the find_peaks
std_buffer = 4 # area around the peak to remove

ga30 = get_multiple_data_arrays(filters=["GaAs_30"])
name = ga30[0][0]
data = ga30[0][1]
channels = np.arange(0, len(data))

In [24]:
# peaks, p_dict = find_peaks(data, prominence=prominence)

In [25]:
peaks, _ = find_peaks(data, prominence=prominence)
fig = go.Figure()
fig.update_layout(title="Plot on channels, find the calibration peaks")
fig.add_trace(go.Scatter(x=channels, y=data, mode="lines", name=name))
for p in peaks:
    fig.add_vline(x=p, line_width=1, line_dash="dash", line_color="red", annotation_text=f"{p}")
# fig

In [26]:
# bounds = ((-np.inf, -3, -np.inf), (np.inf, 3, np.inf))
p0 = [1, peaks[0], 1]
for i in range(1, len(peaks)):
    p0 += [1, peaks[i], 1]

# fit_vals, covar = curve_fit(n_gaussians, channels, data, p0=p0, bounds=bounds)
fit_vals, covar = curve_fit(n_gaussians, channels, data, p0=p0)
fit_amp = fit_vals[0::3]
fit_peaks = fit_vals[1::3]
fit_std = fit_vals[2::3]

In [27]:
fig = go.Figure()
fig.update_layout(title="Fitted peaks <br> <sub>See if a peak is substituded for background in the fitting</sub>")
fig.add_trace(go.Scatter(x=channels, y=data, mode="lines", name=name))
for i in range(len(fit_peaks)):
    fig.add_vline(x=fit_peaks[i], line_width=1, line_dash="dash", line_color="red", annotation_text=f"{np.round(fit_peaks[i], decimals=2):.2f}")
    # add gaussian fit of each peak
    fig.add_trace(go.Scatter(x=channels, y=gaussian(channels, fit_amp[i], fit_peaks[i], fit_std[i]), mode="lines", name=f"fit {fit_peaks[i]:.2f}"))
# fig

In [28]:
# data where the peaks are set to np.nan
bg = data.copy()

def remove_peaks(data, peaks, stds, buffer=3):
    for i in range(len(peaks)):
        data[int((peaks[i] - stds[i] * buffer)) : int((peaks[i] + stds[i] * buffer))] = np.nan
    return data
# for i in range(len(fit_peaks)):
#     bg[int(fit_peaks[i] - fit_std[i]*std_buffer) : int(fit_peaks[i] + fit_std[i]*std_buffer)] = np.nan

# bg[29:34] = np.nan # for GaAs_30 to remove a small section

bg = remove_peaks(bg, fit_peaks, fit_std, buffer=std_buffer)

fig = go.Figure()
fig.update_layout(title="Plot on channels, find the calibration peaks")
fig.add_trace(go.Scatter(x=channels, y=bg, mode="lines", name=name))
for p in fit_peaks:
    fig.add_vline(x=p, line_width=1, line_dash="dash", line_color="red", annotation_text=f"{np.round(p, decimals=2):.2f}")
# fig

In [29]:
# linear background fit under peaks

def remove_bg_linear(data_bg, plot=False):

    while np.isnan(data_bg).any():
        # find the last valid value before the nan
        nan1 = np.where(np.isnan(data_bg))[0][0]
        valid1 = nan1 - 1
        if valid1 == -1:
            valid1 = 0
        # find the first non-nan after the nan
        valid2 = np.where(np.isnan(data_bg[nan1::]) == False)[0][0] + valid1 + 1
        # fit a line between the two points using np.linspace
        y = np.linspace(data_bg[valid1], data_bg[valid2], valid2 - valid1)

        # replace the nan with the fitted line
        data_bg[valid1:valid2] = y
    
    if plot:
        fig = go.Figure()
        fig.update_layout(title="Plot on channels, find the calibration peaks")
        fig.add_trace(go.Scatter(x=channels, y=data_bg, mode="lines", name=name))
        fig.show()
    return data_bg

In [30]:
bg = remove_bg_linear(bg, plot=False)

In [31]:
# count nan
np.isnan(bg).sum()

0

In [32]:
# refine the peak finding, then remove background again
data_wo_bg = data - bg

# fit_vals, covar = curve_fit(n_gaussians, channels, data, p0=p0, bounds=bounds)
fit_vals, covar = curve_fit(n_gaussians, channels, data_wo_bg, p0=p0)
fit_amp = fit_vals[0::3]
fit_peaks = fit_vals[1::3]
fit_std = fit_vals[2::3]

fig = go.Figure()
fig.update_layout(title="Plot on channels, find the calibration peaks")
fig.add_trace(go.Scatter(x=channels, y=data_wo_bg, mode="lines", name=name))
for i in range(len(fit_peaks)):
    fig.add_vline(x=fit_peaks[i], line_width=1, line_dash="dash", line_color="black", annotation_text=f"{np.round(fit_peaks[i], decimals=2):.2f}")
    fig.add_vline(x=fit_peaks[i]-fit_std[i]*3, line_width=0.3, line_dash="dash", line_color="grey")
    fig.add_vline(x=fit_peaks[i]+fit_std[i]*3, line_width=0.3, line_dash="dash", line_color="brown")

    # add gaussian fit of each peak
    fig.add_trace(go.Scatter(x=channels, y=gaussian(channels, fit_amp[i], fit_peaks[i], fit_std[i]), mode="lines", name=f"fit {fit_peaks[i]:.2f}"))
# fig

In [33]:
bg = remove_peaks(bg, fit_peaks, fit_std, buffer=std_buffer)
print(np.isnan(bg).sum())
data_wo_bg = data - remove_bg_linear(bg)

299


In [34]:
fig = go.Figure()
fig.update_layout(title="Peaks after background removal")
fig.add_trace(go.Scatter(x=channels, y=data_wo_bg, mode="lines", name=name))
for i in range(len(fit_peaks)):
    fig.add_vline(x=fit_peaks[i], line_width=1, line_dash="dash", line_color="black", annotation_text=f"{np.round(fit_peaks[i], decimals=2):.2f}")
    fig.add_vline(x=fit_peaks[i]-fit_std[i]*3, line_width=0.3, line_dash="dash", line_color="grey")
    fig.add_vline(x=fit_peaks[i]+fit_std[i]*3, line_width=0.3, line_dash="dash", line_color="brown")

    # add gaussian fit of each peak
    fig.add_trace(go.Scatter(x=channels, y=gaussian(channels, fit_amp[i], fit_peaks[i], fit_std[i]), mode="lines", name=f"fit {fit_peaks[i]:.2f}"))
# fig

In [41]:
bg_fit = np.polyval(np.polyfit(channels, bg, 14), channels)

fig = go.Figure()
fig.update_layout(title=f"Background and backround fit <br> <sub>Background deg={deg}</sub><br> <sub>Std window={std_buffer}</sub>")
fig.add_trace(go.Scatter(x=channels, y=bg, mode="lines", name=f"{name}"))
fig.add_trace(go.Scatter(x=channels, y=bg_fit, mode="lines", name=f"{name} fit"))

# for i in range(len(fit_peaks)):
#     fig.add_vline(x=fit_peaks[i], line_width=1, line_dash="dash", line_color="black", annotation_text=f"{np.round(fit_peaks[i], decimals=2):.2f}")
# fig

's'

's'

In [36]:
# nan in bg
np.isnan(bg).sum()

0

In [40]:
bg_6 = np.polyval(np.polyfit(channels, bg, 6), channels)
bg_14 = np.polyval(np.polyfit(channels, bg, 14), channels)
bg_20 = np.polyval(np.polyfit(channels, bg, 20), channels)


fig = go.Figure()
fig.update_layout(title=f"Background and backround fit <br> <sub>Background deg={deg}</sub><br> <sub>Std window={std_buffer}</sub>")
fig.add_trace(go.Scatter(x=channels, y=bg, mode="lines", name=f"bg"))
fig.add_trace(go.Scatter(x=channels, y=bg_6, mode="lines", name=f"bg_6"))
fig.add_trace(go.Scatter(x=channels, y=bg_14, mode="lines", name=f"bg_14"))
fig.add_trace(go.Scatter(x=channels, y=bg_20, mode="lines", name=f"bg_20"))

fig.add_trace(go.Scatter(x=channels, y=bg_fit, mode="lines", name=f"{name} fit"))

# for i in range(len(fit_peaks)):
#     fig.add_vline(x=fit_peaks[i], line_width=1, line_dash="dash", line_color="black", annotation_text=f"{np.round(fit_peaks[i], decimals=2):.2f}")
# fig

'f'

c:\Users\Brynjar\miniconda3\envs\hspy\lib\site-packages\IPython\core\interactiveshell.py:3398: RankWarning:

Polyfit may be poorly conditioned



'f'

In [38]:
np.polyfit(channels, bg, 14)

array([-1.51503617e-43,  2.22449020e-39, -1.48999016e-35,  6.05110351e-32,
       -1.66873457e-28,  3.30762309e-25, -4.84210652e-22,  5.26572597e-19,
       -4.19822415e-16,  2.37527691e-13, -9.01663220e-11,  2.10075730e-08,
       -2.67126355e-06,  1.98221641e-04, -1.10758217e-03])

In [39]:
np.polyfit(channels, bg, 6)

array([ 9.05554126e-21, -4.70306016e-17,  6.78761533e-14,  2.60345935e-11,
       -1.35154252e-07,  8.67934735e-05, -7.71957316e-04])